# Installing required dependencies

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

print(tf.__version__)

"""
hyperparameters are:

Number of nodes 
epochs
learning rate

"""

In [ ]:
window_size = 30
batch_size = 32
shuffle_buffer_size = 1000

# Notes
 before running below codes, make sure you have csv file of sunspots which is to be used at current location i.e. location same as this notebook




In [ ]:
!ls /kaggle/input/sunspots/Sunspots.csv

In [ ]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.title('Forecasting trend')
    plt.xlabel('Date')
    plt.ylabel('Monthly Mean Total Sunspot Number')
    plt.grid(True)

In [ ]:
file_path = '/kaggle/input/sunspots/Sunspots.csv'
df = pd.read_csv(file_path,usecols=['Date','Monthly Mean Total Sunspot Number'])
df.dropna(inplace=True)

In [ ]:
time_data = df['Date'].to_numpy()
series = df['Monthly Mean Total Sunspot Number'].to_numpy()

In [ ]:
len(time_data)

# getting data testing purpose

data from data from **Aug 2018** to **Dec 2019** will be used for the purpose of **Testing** out Forecasting model (as per assignment)




In [ ]:
test_df = df[(df['Date'] > '2018-08-01') & (df['Date'] <= '2019-12-31')] # date from aug 2018 to dec 2019 for testing 

In [ ]:
test_df.tail()

# Visualizing Forecasting Data

In [ ]:
plt.figure(figsize=(20,10))
plt.title('Forecasting trend')
plt.xlabel('Date')
plt.ylabel('Monthly Mean Total Sunspot Number')

plt.plot(df['Date'], df['Monthly Mean Total Sunspot Number'])
plt.show()

# splitting series  data for train and validation datasets

In [ ]:
split_time = 3000
time_train = time_data[:split_time]
x_train = series[:split_time]

time_valid = time_data[split_time:]
x_valid = series[split_time:]



# Windowing Forecasting data 

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    """
    this function creates windows from inputted series, like this fashion:
    1: [1,2,3,4]. --> [4,3,2,1]
    2: [2,3,4,5]. --> [3,4,5,2]
    3: [3,4,5,6]. --> [6,3,5,4]
    4: [6,7,8,9]...

    """
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=2, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [ ]:
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

## 1) Defining Simple DNN based sequential Network

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, input_shape=[window_size], activation="relu"), # to avoid vanishing gradient problem
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1)
])

model.compile(loss='mse', optimizer=tf.keras.optimizers.SGD(lr=1e-7, momentum=0.9))


In [ ]:
model.summary()

# Training of Deep Neural Network

In [ ]:
%%time
history = model.fit(dataset, epochs=100, verbose=0)

In [ ]:
%%time 
forecast = []

for time in range(len(series) - window_size):
    forecast.append(model.predict(series[time:time + window_size][np.newaxis]))

forecast = forecast[split_time-window_size:]
results = np.array(forecast)[:, 0, 0]

In [ ]:
# Visualising the result from Aug 2018 to Dec 2019
plt.figure(figsize=(20,10))
plot_series(time_valid[-len(test_df):], x_valid[-len(test_df):])
plot_series(time_valid[-len(test_df):], results[-len(test_df):])

In [ ]:
# Visualising on whole dataset
plt.figure(figsize=(20,10))
plot_series(time_valid, x_valid)
plot_series(time_valid, results)
plt.show()

In [ ]:
len(x_valid)

In [ ]:
# Evaluate the model using MAE
tf.keras.metrics.mean_absolute_error(x_valid, results).numpy()

# 2) LSTM based Architecture 

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(0)
np.random.seed(0)

# Prepare the training set
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

dataset = windowed_dataset(x_train, window_size=60, batch_size=64, shuffle_buffer=shuffle_buffer_size)

# Define the model
model_1 = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=60, kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.Dense(80, activation="relu"),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 400)
])
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
opt1 = tf.keras.optimizers.SGD(lr=1e-5)

model_1.compile(loss=tf.keras.losses.Huber(),
              optimizer=opt1,
              metrics=["mae"])

In [ ]:
model_1.summary()

# training of LSTM Arch.



In [ ]:
%%time
history = model_1.fit(dataset, epochs=100, verbose=1)

# forecasting Predicted output on LSTM Arch.
---


In [ ]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [ ]:
# Predict the result
lstm_rnn_forecast = model_forecast(model_1, series[..., np.newaxis], window_size)
lstm_rnn_forecast = lstm_rnn_forecast[split_time - window_size:-1, -1, 0]

In [ ]:
# Visualising the result from Aug 2018 to Dec 2019
plt.figure(figsize=(20,10))
plot_series(time_valid[-len(test_df):], x_valid[-len(test_df):])
plot_series(time_valid[-len(test_df):], lstm_rnn_forecast[-len(test_df):])

In [ ]:
# Visualising on whole dataset
plt.figure(figsize=(20,10))
plot_series(time_valid, x_valid)
plot_series(time_valid, lstm_rnn_forecast)
plt.show()

In [ ]:
# Evaluate the model using MAE
tf.keras.metrics.mean_absolute_error(x_valid, lstm_rnn_forecast).numpy()

# 3) Simple RNN based Architecture



In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(0)
np.random.seed(0)

def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

dataset = windowed_dataset(x_train, window_size=60, batch_size=64, shuffle_buffer=shuffle_buffer_size)

# Define the model

model_2 = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(64, return_sequences=True, activation="relu", input_shape=[None, 1]),
    tf.keras.layers.SimpleRNN(64, return_sequences=True, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1),
])


model_2.compile(loss=tf.keras.losses.Huber(),
              optimizer='adam',
              metrics=["mae"])

model_2.summary()

In [ ]:
%%time 
history = model_2.fit(dataset, epochs=100, verbose=1)

In [ ]:
# Predict the result
rnn_forecast = model_forecast(model_2, series[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

In [ ]:
# Visualising the result
plt.figure(figsize=(20,10))
plot_series(time_valid[-len(test_df):], x_valid[-len(test_df):])
plot_series(time_valid[-len(test_df):], rnn_forecast[-len(test_df):])
plt.show()

In [ ]:
# Visualising on whole dataset
plt.figure(figsize=(20,10))
plot_series(time_valid, x_valid)
plot_series(time_valid, rnn_forecast)
plt.show()

In [ ]:
# Evaluate the model using MAE
tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()